# PS3: Primal and Dual Flux Balance Analysis of the Urea Cycle in HL-60 Cells
The [urea cycle](https://www.kegg.jp/pathway/hsa00220) is a crucial metabolic pathway that converts toxic ammonia into urea for excretion. While the urea cycle's role in [HL-60 cells, a human promyelocytic leukemia cell line](https://www.atcc.org/products/ccl-240?matchtype=b&network=g&device=c&adposition=&keyword=hl60%20cell%20line%20atcc&gad_source=1&gbraid=0AAAAADR6fpoOXsp8U8fXLd_E6sLTcwv24&gclid=CjwKCAiA5eC9BhAuEiwA3CKwQm0C1oE5_JjTpJ24VnTjZUZQVLivpPxmufDo7HdH5v3hN1XKnEf3ExoCvhwQAvD_BwE), is not directly established, these cells exhibit alterations in protein levels and proliferation rates when exposed to various compounds, which may indirectly affect nitrogen metabolism and related pathways.

In this lab, we will explore a simplified model of the urea cycle in HL-60 cells using flux balance analysis (FBA). [Flux balance analysis (FBA)](https://pubmed.ncbi.nlm.nih.gov/20212490/) enables researchers to _estimate_ metabolic fluxes and optimize cellular metabolism, providing insights into biological systems' operation without requiring extensive kinetic parameter information. 

__Reference:__
[Orth JD, Thiele I, Palsson BØ. What is flux balance analysis? Nat Biotechnol. 2010 Mar;28(3):245-8. doi: 10.1038/nbt.1614. PMID: 20212490; PMCID: PMC3108565](https://pubmed.ncbi.nlm.nih.gov/20212490/)

> __Learning Objectives:__
>
> By the end of this lab, you should be able to:
> - __Primal flux balance analysis (FBA)__: Understand the principles of flux balance analysis and how to formulate it as a linear programming problem, including estimation of reaction parameters using biological databases like eQuilibrator and BRENDA.
> - __Dual optimization theory__: Learn to formulate and solve the dual problem associated with flux balance analysis, understanding the mathematical relationship between primal and dual solutions in metabolic optimization.
> - __Metabolic bottleneck analysis__: Analyze both primal and dual FBA results to interpret metabolic fluxes, identify rate-limiting steps through dual variable analysis, and evaluate system behavior under different constraints.

FBA is a very important tool in systems biology and metabolic engineering. It's also a clever application of linear programming. Let's get started!
___

## Motivation
[Flux balance analysis (FBA)](https://pubmed.ncbi.nlm.nih.gov/20212490/) enables researchers to _estimate_ metabolic fluxes and optimize cellular metabolism, providing insights into biological systems' operation without requiring extensive kinetic parameter information. FBA can operate in data-rich and data-poor situations.
* __Structure__. The flux balance analysis problem is a linear program composed of an objective (what we are trying to optimize), constraints (the rules of the world, in our case material balances), and bounds (limits on the decision variables). It returns the optimal values for the reaction rates in a system (metabolic fluxes).
* __Integration__. Flux balance analysis integrates measurement data (extracellular uptake, intracellular omics, etc.) using a model of the global operation of a cell. FBA provides a _snapshot_ of the state of a system. While more data enhances realism, FBA has low data requirements.

Flux balance analysis (FBA) has some _perceived_ limitations.
* __Not unique__. FBA does not specify fluxes in a metabolic network uniquely, as regulatory mechanisms affecting enzyme kinetics and expression influence the chosen flux distribution, resulting in multiple potential flux solutions for an optimal state. This limitation is __major__ (and true).
* __Not dynamic__. FBA cannot model dynamic metabolic behavior due to its steady-state assumption, limiting its ability to capture temporal changes. However, [we can adapt FBA to be approximately dynamic](https://pmc.ncbi.nlm.nih.gov/articles/PMC1302231/), making this limitation minor.
* __No regulation__. FBA may conflict with experimental data, especially when regulatory loops are excluded. These discrepancies reveal the limitations of relying only on stoichiometric information without considering complex cellular regulation. This can be fixed with [regulatory flux balance analysis](https://pubmed.ncbi.nlm.nih.gov/11708855/). Gene expression is _easy(ish)_, but allosteric regulation (activity) is hard.

Example FBA publications:
* [Edwards JS, Ibarra RU, Palsson BO. In silico predictions of Escherichia coli metabolic capabilities are consistent with experimental data. Nat Biotechnol. 2001 Feb;19(2):125-30. doi: 10.1038/84379. PMID: 11175725.](https://pubmed.ncbi.nlm.nih.gov/11175725/)
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://pubmed.ncbi.nlm.nih.gov/29944340/)
* [Tan ML, Jenkins-Johnston N, Huang S, Schutrum B, Vadhin S, Adhikari A, Williams RM, Zipfel WR, Lammerding J, Varner JD, Fischbach C. Endothelial cells metabolically regulate breast cancer invasion toward a microvessel. APL Bioeng. 2023 Dec 4;7(4):046116. doi: 10.1063/5.0171109. PMID: 38058993; PMCID: PMC10697723.](https://pubmed.ncbi.nlm.nih.gov/38058993/)

___

## Background
Flux balance analysis (FBA) is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where metabolite production, consumption, and transport rates are balanced.

> To formulate the FBA problem, we first define the key sets and variables:
>- $\mathcal{R}$: the set of biochemical reactions in the network.
>- $\mathcal{M}$: the set of chemical species (metabolites) involved in the reactions.
>- $\hat{v}_i$: the flux (rate) through reaction $i \in \mathcal{R}$, which we want to estimate.
>- $c_i$: the objective coefficient for reaction $i$, chosen by the user to define what to optimize (e.g., maximize or minimize).

The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. The linear program is:
$$
\boxed{
\begin{align*}
\max_{\hat{v}}\quad&  \sum_{i\in\mathcal{R}}c_{i}\hat{v}_{i}\\
\text{subject to}\quad & \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j} = 0\qquad\forall{i\in\mathcal{M}}\\
& \mathcal{L}_{j}\leq\hat{v}_{j}\leq\mathcal{U}_{j}\qquad\forall{j\in\mathcal{R}}
\end{align*}}
$$
Here, $\sigma_{ij}$ are elements of the stoichiometric matrix $\mathbf{S}$, and $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ are the lower and upper flux bounds for reaction $j$, respectively. Buried in the material constraints, there are many assumptions. To see where these material balance constraints come from, see the [derivation notebook](./CHEME-5800-L6b-Derivation-Constraints-FBA-Fall-2025.ipynb).

### Constraints: Stoichiometric matrix
Suppose we have a set of biochemical reactions $\mathcal{R}$ involving chemical species (metabolite) set $\mathcal{M}$. Then, the stoichiometric matrix is a $\mathbf{S}\in\mathbb{R}^{|\mathcal{M}|\times|\mathcal{R}|}$ matrix, where $|\mathcal{M}|$ denotes the number of chemical species and $|\mathcal{R}|$ denotes the number of reactions. The elements of the stoichiometric matrix $\sigma_{ij}\in\mathbf{S}$ are stoichiometric coefficients such that:
* $\sigma_{ij}>0$: Chemical species (metabolite) $i$ is _produced_ by reaction $j$. Species $i$ is a product of reaction $j$.
* $\sigma_{ij} = 0$: Chemical species (metabolite) $i$ is not connected with reaction $j$.
* $\sigma_{ij}<0$: Chemical species (metabolite) $i$ is _consumed_ by reaction $j$. Species $i$ is a reactant of reaction $j$.

The stoichiometric matrix $\mathbf{S}$ is the digital representation of the biochemistry occurring inside some volume, i.e., inside the cell, in a test tube in the case of cell-free systems, or some abstract volume such as a compartment or pseudocompartment of interest.

### Bounds: Models for flux bounds
The flux bounds $\mathcal{L}_j$ and $\mathcal{U}_j$ are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric matrix. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the problem. 

A general model for these bounds is given by:
$$
\begin{align*}
-\delta_{j}\left[{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}\right]\leq\hat{v}_{j}\leq{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}
\end{align*}
$$
where:
- $V_{max,j}^{\circ}$ denotes the maximum reaction velocity (units: flux) computed at some _characteristic enzyme abundance_.
- $e$ is the actual enzyme abundance, and $e^{\circ}$ is the characteristic enzyme abundance (units: concentration).
- $\theta_{j}(\dots) \in [0,1]$ is the current fraction of maximal enzyme activity for the enzyme in reaction $j$, describing [allosteric effects](https://en.wikipedia.org/wiki/Allosteric_regulation).
- $f_j(\dots)$ is a function describing the substrate dependence of the reaction rate (units: dimensionless).
- $\delta_j \in \{0,1\}$ is the binary direction parameter indicating reversibility: $\delta_j = 1$ if reaction $j$ is reversible, $\delta_j = 0$ if irreversible.

Thus, the maximum reaction velocity is given by:
$$
V_{max,j}^{\circ} = k_{cat,j}^{\circ} e^{\circ}
$$
where $k_{cat,j}$ is the catalytic constant or turnover number for the enzyme (units: 1/time).

> __Parameters__: In general, we need estimates for the $k_{cat,j}^{\circ}$ for all enzymes in the system and a _reasonable policy_ for specifying a characteristic value for $e^{\circ}$. In addition, the $\theta_{j}(\dots)$ and $f_{j}(\dots)$ models can also have associated parameters, e.g., saturation or binding constants, etc. Thus, we need to estimate these from literature studies or experimental data.

Let's simplify this model to estimate the flux bounds for our urea cycle model.

### Simplified Bounds Model
In this lab, we will use a simplified bounds model to estimate the flux bounds.
We assume that $(e/e^{\circ}) \sim 1$, there are no allosteric effects $\theta_{j}(\dots) \sim 1$, and the substrates are saturating $f_{j}(\dots) \sim 1$.
Then, the flux bounds are given by:
$$
\begin{align*}
-\delta_{j} V_{max,j}^{\circ} \leq \hat{v}_{j} \leq V_{max,j}^{\circ}
\end{align*}
$$
We can see that the flux bounds are a function of the maximum reaction velocity, the catalytic constant or turnover number, and our assumed value for the characteristic enzyme abundance.

___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, and more. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl"));

<div>
    <center>
        <img src="figs/Fig-Urea-cycle-Schematic.png" width="680"/>
    </center>
</div>

### Build the model

To store all the problem data, we created [the `MyPrimalFluxBalanceAnalysisCalculationModel` type](src/Types.jl). Let's build one of these objects for our problem and store it in the `model::MyPrimalFluxBalanceAnalysisCalculationModel` variable. We also return the `rd::Dict{String, String}` dictionary, which maps the reaction name field (key) to the reaction string (value).

> __Builder (or factory) pattern__: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to make that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.

Let's build our model:

In [2]:
primal_model, rd = let

    # first, load the reaction file and process it
    list_of_reactions = read_reaction_file(joinpath(_PATH_TO_DATA, "Network.net")); # load the reactions from the VFF reaction file
    S, species, reactions, rd = build_stoichiometric_matrix(list_of_reactions); # Builds the stoichiometric matrix, species list, and the reactions list
    bounds_array = build_default_bounds_array(list_of_reactions); # Builds a default bounds model using the flat file flags

    # build the FBA model -
    model = build(MyPrimalFluxBalanceAnalysisCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = bounds_array, # these are the *default* bounds, we'll need to update with new info if we have it
        species = species, # list of species. The rows of S are in this order
        reactions = reactions, # list of reactions. The cols of S are in this order
        objective = length(reactions) |> R -> zeros(R), # this is empty, we'll need to set this
    ));

    # return -
    model, rd
end;

In [3]:
rd # let's look at the reaction dictionary

Dict{String, String} with 19 entries:
  "v4"  => "M_Carbamoyl_phosphate_c+M_L-Ornithine_c = M_Orthophosphate_c+M_L-Ci…
  "v5"  => "2*M_L-Arginine_c+4*M_Oxygen_c+3*M_NADPH_c+3*M_H_c = 2*M_Nitric_oxid…
  "v1"  => "M_ATP_c+M_L-Citrulline_c+M_L-Aspartate_c = M_AMP_c+M_Diphosphate_c+…
  "b12" => "[] = M_Nitric_oxide_c"
  "b2"  => "[] = M_L-Aspartate_c"
  "b14" => "[] = M_H2O_c"
  "b8"  => "[] = M_Orthophosphate_c"
  "b11" => "[] = M_H_c"
  "v3"  => "M_L-Arginine_c+M_H2O_c = M_L-Ornithine_c+M_Urea_c"
  "b7"  => "[] = M_Diphosphate_c"
  "b9"  => "[] = M_Oxygen_c"
  "b3"  => "[] = M_Fumarate_c"
  "b5"  => "[] = M_ATP_c"
  "b10" => "[] = M_NADPH_c"
  "v2"  => "M_N-(L-Arginino)succinate_c = M_Fumarate_c+M_L-Arginine_c"
  "b4"  => "[] = M_Urea_c"
  "b6"  => "[] = M_AMP_c"
  "b13" => "[] = M_NADP_c"
  "b1"  => "[] = M_Carbamoyl_phosphate_c"

___

## Task 1: Formulate the Flux Bounds Array
In this task, we formulate the flux bounds array. The flux bounds are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric matrix. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the problem. 

First, let's estimate the reversibility parameter $\delta_{j}$ for each of the five enzyme-catalyzed reactions in our model of the urea cycle using [eQuilibrator](https://equilibrator.weizmann.ac.il).
* [Beber ME, Gollub MG, Mozaffari D, Shebek KM, Flamholz AI, Milo R, Noor E. eQuilibrator 3.0: a database solution for thermodynamic constant estimation. Nucleic Acids Res. 2022 Jan 7;50(D1): D603-D609. doi: 10.1093/nar/gkab1106. PMID: 34850162; PMCID: PMC8728285.](https://pubmed.ncbi.nlm.nih.gov/34850162/)

`TODO`: Fill in the missing elements in the code block below to complete this task; when using [eQuilibrator](https://equilibrator.weizmann.ac.il), use the EC prefix, i.e., `EC 6.3.4.5` and the mitochondrial (superscript `m`) values.

Store the results of this analysis in the `reversibility_parameter_dictionary::Dict{String, Int}` dictionary, which maps the reaction name field (key) to the estimated reversibility parameter (value). Use a $\Delta\bar{G}$ threshold value (hyperparameter) of -10.0 kJ/mol to determine the reversibility of the reactions.

In [4]:
reversibility_parameter_dictionary = let

    # alias the model for clarity
    model = primal_model; 

    # initialize -
    ΔḠ = -10.0; # threshold value, units: kJ/mol -
    names = model.reactions; # get an array of the names of the reactions (includes exchange)
    reversibility_parameter_dictionary = Dict{String, Int}();

    # TODO: build a ΔG array for the reactions in the model
    ΔG = [
        -4.3 ; # 1 v₁ EC 6.3.4.5 value: -4.3 ± 2.9 kJ/mol
        -5.5 ; # 2 v₂ EC 4.3.2.1 value: -5.5 ± 5.7 kJ/mol
        -51.0 ; # 3 v₃ EC 3.5.3.1 value: -51 ± 12.4 kJ/mol
        -30.3 ; # 4 v₄ EC 2.1.3.3 value: -30.3 ± 5.7 kJ/mol
        -1220.2 ; # 5 v₅ EC 1.15.13.39 value: -1220.2 ± 29.6 kJ/mol # TODO: DQ2 What if this was +1210?
    ];

    # compute loop -
    for i ∈ eachindex(names)
        name = names[i]; # get the reaction name for reaction i -

        # check: do we have an exchange flux? If so: skip
        if (contains(name, "b") == true)
            continue;
        end
        
        ΔGᵢ = ΔG[i]; # get the ΔG value for reaction i -
        
        # compute
        δᵢ = sign(ΔGᵢ -  ΔḠ) == 1 ? 1 : 0 # ternary operator: if sign(ΔΔG) == 1 then reversible (1), else irreversible (0)
        
        # store -
        reversibility_parameter_dictionary[name] = δᵢ; # stores the reversibility parameter with key name
    end

    
    # return -
    reversibility_parameter_dictionary;
end;

### Estimate the maximum reaction velocity $V_{max,j}^{\circ}$ for reaction $j$
Next, we'll estimate the maximum reaction velocity $V_{max,j}^{\circ}$ for each of the 5 enzyme-catalyzed reactions in our model of the Urea cycle using [BRENDA](https://www.brenda-enzymes.org/):
* [Antje Chang et al., BRENDA, the ELIXIR core data resource in 2021: new developments and updates, Nucleic Acids Research, Volume 49, Issue D1, 8 January 2021, Pages D498–D508, https://doi.org/10.1093/nar/gkaa1025](https://academic.oup.com/nar/article/49/D1/D498/5992283)

Assume the characteristic enzyme abundance $e^{\circ}\simeq$ 0.01 `mmol/gDW` for all reactions. For missing turnover numbers, use a characteristic value of $k_{cat,j}^{\circ}\simeq$ 10 `1/s`. Store the results of this analysis in the `maximum_reaction_velocity_dictionary::Dict{String, Float64}` dictionary, which maps the reaction name field (key) to the estimated maximum reaction velocity (value). 

`TODO`: Fill in the missing elements in the code block below to complete this task:

In [5]:
maximum_reaction_velocity_dictionary = let

    # alias the model for clarity
    model = primal_model;

    # initialize -
    eₒ = 0.01; # characteristic enzyme abundance units: mmol/gDW
    kₒ = 10.0; # characteristic turnover rate units: 1/s (use this if we don't have a specific value from BRENDA)
    names = model.reactions; # get an array of the names of the reactions (includes exchange)
    maximum_reaction_velocity_dictionary = Dict{String, Float64}();

    # TODO: **Update** the kcat array for the reactions in the model with values from BRENDA
    kcat = [
        kₒ ; # 1 v₁ EC 6.3.4.5 No value, use default
        3.28 ; # 2 v₂ EC 4.3.2.1 value: 3.28 1/s
        190.0 ; # 3 v₃ EC 3.5.3.1 value: 190 1/s
        410.0 ; # 4 v₄ EC 2.1.3.3 value: 410 1/s (E. coli value)
        kₒ ; # 5 v₅ EC 1.15.13.39 No value, use default
    ];

    # compute loop -
    for i ∈ eachindex(names)
        name = names[i]; # get the reaction name for reaction i -

        # check: do we have an exchange flux? If so: skip
        if (contains(name, "b") == true)
            continue;
        end
        
        # compute the VMaxᵢ -
        VMaxᵢ = kcat[i]*eₒ;

        # store -
        maximum_reaction_velocity_dictionary[name] = VMaxᵢ; # stores the VMax with key name
    end
    
    # return -
    maximum_reaction_velocity_dictionary;
end;

Let's make a table that holds the Vmax and reversibility parameter for each reaction in our model:

In [6]:
let

    # initialize -
    df = DataFrame();
    sorted_reaction_names = maximum_reaction_velocity_dictionary |> keys |> collect |> sort; # get the keys and sort them

    for name ∈ sorted_reaction_names
        VMax = maximum_reaction_velocity_dictionary[name]; # get the VMax value
        push!(df, (
            Reaction = name, 
            VMax = VMax,
            isreversible = reversibility_parameter_dictionary[name] == 1 ? true : false, # ternary operator
        )); # push to the dataframe
    end
   
    df |> data -> pretty_table(data, backend = :text,
         table_format = TextTableFormat(borders = text_table_borders__compact))
end

 ---------- --------- --------------
  Reaction      VMax   isreversible 
    String   Float64           Bool 
 ---------- --------- --------------
        v1       0.1           true
        v2    0.0328           true
        v3       1.9          false
        v4       4.1          false
        v5       0.1          false
 ---------- --------- --------------


### Update the flux bounds array
Now that we have the reversibility parameters and the maximum reaction velocities, we can update the flux bounds for each reaction in our model of the Urea cycle. 

In [7]:
fluxbounds = let
    
    # alias the model for clarity
    model = primal_model;

    # update the flux bounds -
    fluxbounds = model.fluxbounds;
    names = model.reactions;
    for i ∈ eachindex(names)
        name = names[i]; # get the reaction name for reaction i -
    
        # check: do we have an exchange flux? If so: skip
        if (contains(name, "b") == true)
            continue;
        end
        
        VMax = maximum_reaction_velocity_dictionary[name]; # what is the maximum velocity for this reaction?
        δᵢ = reversibility_parameter_dictionary[name]; # what is the reversibility parameter for this reaction?
    
        # update the bounds
        fluxbounds[i,1] = -δᵢ*VMax; # lower bound (negative for reversible reactions)
        fluxbounds[i,2] = VMax; # upper bound
    end

    # return -
    fluxbounds;
end;

In [8]:
primal_model.fluxbounds = fluxbounds; # update the model with the new flux bounds

### Update the objective function
Finally, let's update the objective function of our model. By default, all the elements of the objective function are set to `0.0`. Update the objective function to _maximize_ the export of  `Urea` from the system. 

`TODO`: Fill in the missing elements in the code block below to complete this task:

In [9]:
objective = primal_model.objective;
reaction_to_maximize = "b4"; # TODO: specify the reaction to maximize (use the reaction name)
findfirst(x-> x==reaction_to_maximize, primal_model.reactions) |> i -> objective[i] = -1; # TODO: why negative?

## Task 2: Compute the optimal primal flux distribution
Let's compute the optimal metabolic distribution $\left\{\hat{v}_{i} \mid i = 1,2,\dots,\mathcal{R}\right\}$ by solving the [linear programming problem](https://en.wikipedia.org/wiki/Linear_programming). We solve the optimization problem by passing the `model::MyPrimalFluxBalanceAnalysisCalculationModel` to [the `solve(...)` method](src/Compute.jl). 

> __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method will [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

The [`solve(...)` method](src/Compute.jl) returns a `primal_solution::Dict{String, Any}` dictionary containing information about the solution.

In [10]:
primal_solution = let
    
    model = primal_model; # alias the model for clarity
    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oops! Looks like we have a major malfunction - the problem didn't solve
    end

    # return solution
    solution
end;

### Flux table
The code block below shows how we constructed the flux table using [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl).
> __Summary__: Each row of the flux table holds information about a reaction in the model. The first column has the reaction name, the second column has the estimated optimal flux value (solution of the FBA problem), and the third column has the reaction string for the flux.

So what do we get?

In [11]:
let

    # alias the model and solution for clarity
    model = primal_model;
    solution = primal_solution;

    # initialize -
    S = model.S;
    flux_bounds_array = model.fluxbounds;
    number_of_reactions = size(S,2); # columns
    flux = solution["argmax"];
    names = model.reactions; # get an array of the names of the reactions (includes exchange)
    df = DataFrame();
    
    # populate the state table -
	for reaction_index = 1:number_of_reactions
        row_df = (
            i = reaction_index,
            name = names[reaction_index],
            flux = flux[reaction_index],
            reaction = model.reactions[reaction_index] |> key-> rd[key]
        )

		push!(df, row_df)
	end
	
	# write the table -
	pretty_table(df, backend = :text, auto_wrap = true, 
        alignment_anchor_fallback = :l,
        alignment_anchor_regex = [4 => [r""]],
        fit_table_in_display_horizontally = false,
        fit_table_in_display_vertically = false,
        apply_alignment_regex_to_summary_rows = true,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end


 ------- -------- --------- ----------------------------------------------------------------------------------------------------------------
      i     name      flux                                                                                                         reaction 
  Int64   String   Float64                                                                                                           String 
 ------- -------- --------- ----------------------------------------------------------------------------------------------------------------
      1       v1    0.0328   M_ATP_c+M_L-Citrulline_c+M_L-Aspartate_c = M_AMP_c+M_Diphosphate_c+M_N-(L-Arginino)succinate_c
      2       v2    0.0328   M_N-(L-Arginino)succinate_c = M_Fumarate_c+M_L-Arginine_c
      3       v3    0.0328   M_L-Arginine_c+M_H2O_c = M_L-Ornithine_c+M_Urea_c
      4       v4    0.0328   M_Carbamoyl_phosphate_c+M_L-Ornithine_c = M_Orthophosphate_c+M_L-Citrulline_c
      5       v5       0.0   2*M_L-Argi

In [12]:
do_I_see_the_primal_flux_table = nothing; # TODO: update this flag to {true | false} if the flux table is visible

## Task 3: Solve the dual problem
In this task, we will solve the dual problem associated with our primal flux balance analysis problem. However, there is a wrinkle. The right-hand side of the primal problem is zero, which means we have a special dual problem called the Fenchel dual which takes the form:
$$
\boxed{
\begin{align*}
\min_{y,p,q}\quad& \mathbf{\mathcal{U}}^{\top}\mathbf{p} - \mathcal{L}^{\top}\mathbf{q}\\
\text{subject to}\quad & \mathbf{S}^{\top}\mathbf{y} + \mathbf{p} - \mathbf{q} = \mathbf{c}\\
& y_{i}\text{ free}\qquad\forall{i\in\mathcal{M}}\\
& q_{j}\geq 0\qquad\forall{j\in\mathcal{R}}\\
& p_{j}\geq 0\qquad\forall{j\in\mathcal{R}}
\end{align*}}
$$
where $\mathbf{S}^{\top}$ is the transpose of the stoichiometric matrix, $\mathbf{y}\in\mathbb{R}^{|\mathcal{M}|}$ is the vector of dual variables associated with the material balance constraints, $\mathbf{p}\in\mathbb{R}^{|\mathcal{R}|}$ is the vector of dual variables associated with the upper flux bounds, and $\mathbf{q}\in\mathbb{R}^{|\mathcal{R}|}$ is the vector of dual variables associated with the lower flux bounds, and $\mathbf{c}\in\mathbb{R}^{|\mathcal{R}|}$ is the vector of (primal) objective coefficients. 

We have all the data we need to solve the dual problem, however, we need to make a few changes. Let's start by creating the bounds arrays for the dual problem.
> __Dual bounds arrays__: The dual bounds arrays will be a $(2|\mathcal{R}| + |\mathcal{M}|) \times {2}$ array, where each row corresponds to a dual variable, the first column is the lower bound, while the second column corresponds to the upper bound on the dual variables. 
> 
> Let's repackage our dual variables into a single array $\hat{\mathbf{z}}\in\mathbb{R}^{(2|\mathcal{R}| + |\mathcal{M}|)}$ such that: $\hat{\mathbf{z}} = [\mathbf{y}; \mathbf{p}; \mathbf{q}]$. The first $|\mathcal{M}|$ elements of $\hat{\mathbf{z}}$ correspond to the dual variables associated with the material balance constraints ($\mathbf{y}$), the next $|\mathcal{R}|$ elements correspond to the dual variables associated with the upper flux bounds ($\mathbf{p}$), and the last $|\mathcal{R}|$ elements correspond to the dual variables associated with the lower flux bounds ($\mathbf{q}$).

Let's build the dual bounds array:

In [13]:
dual_bounds_array = let

    # alias the model for clarity
    model = primal_model;

    # initialize -
    S = model.S; # get the stoichiometric matrix
    number_of_reactions = size(S,2); # columns |R|
    number_of_metabolites = size(S,1); # rows |M|
    number_of_dual_variables = 2*number_of_reactions + number_of_metabolites;
    dual_bounds_array = zeros(number_of_dual_variables, 2); # initialize the dual bounds array
    
    # TODO: populate the dual bounds array
    # TODO: After you finish populating the dual bounds array, comment out the line below
    throw(ErrorException("TODO: populate the dual bounds array"));

    dual_bounds_array; # return
end;

ErrorException: TODO: populate the dual bounds array

Next, we need to build the objective function for the dual problem. The objective function for the dual problem will correspond to the upper and lower flux bounds of the primal problem. However, there's an additional complexity! 

> __Dual objective function__: The dual objective function will be a $(2|\mathcal{R}| + |\mathcal{M}|)$ array, where each element corresponds to a dual variable. The first $|\mathcal{M}|$ elements of the dual objective will be zero (correspond to $\mathbf{y}$), the next $|\mathcal{R}|$ elements will correspond to the upper flux bounds of the primal problem (correspond to $\mathbf{p}$), and the last $|\mathcal{R}|$ elements will correspond to the negative of the lower flux bounds of the primal problem (correspond to $\mathbf{q}$).

Let's build the dual objective function array:

In [14]:
dual_objective_coefficient_array = let

    # alias the model for clarity
    model = primal_model;

    # initialize -
    S = model.S; # get the stoichiometric matrix
    flux_bounds_array = model.fluxbounds; # get the flux bounds array
    number_of_reactions = size(S,2); # columns |R|
    number_of_metabolites = size(S,1); # rows |M|
    number_of_dual_variables = 2*number_of_reactions + number_of_metabolites;
    dual_objective_coefficient_array = zeros(number_of_dual_variables); # initialize the dual objective function array

    # TODO: populate the dual objective function array
    # TODO: After you populate the dual objective function array, comment out the line below
    throw(ErrorException("TODO: populate the dual objective function array"));

    dual_objective_coefficient_array; # return
end;

ErrorException: TODO: populate the dual objective function array

Next, we need to build the constraint matrix for the dual problem. 
> __Constraint Matrix Dual Problem:__ The constraint matrix for the dual problem will be a block matrix composed of three submatrices. The first submatrix will correspond to the transpose of the stoichiometric matrix of the primal problem, the second submatrix will be an identity matrix of size $|\mathcal{R}|\times|\mathcal{R}|$, and the third submatrix will be a negative identity matrix of size $|\mathcal{R}|\times|\mathcal{R}|$. Thus, the constraint matrix for the dual problem is $[\mathbf{S}^{\top}, \mathbf{I}, -\mathbf{I}]$.

Let's build the dual constraint matrix:

In [15]:
A = let

    # alias the model for clarity
    model = primal_model;

    # initialize -
    S = model.S; # get the stoichiometric matrix
    number_of_reactions = size(S,2); # columns |R|
    number_of_metabolites = size(S,1); # rows |M|

    # build the block matrix A -
    ST = transpose(S) |> Matrix{Float64}; # transpose of the stoichiometric matrix
    IR = diagm(0 => ones(number_of_reactions)) # identity matrix for reactions

    # TODO: build the block matrix A
    # TODO: After you build the block matrix A, comment out the line below
    throw(ErrorException("TODO: build the block matrix A"));

    A; # return
end

ErrorException: TODO: build the block matrix A

Now, we are ready to set up the dual problem and solve it. We have a type for the dual problem called [the `MyDualFluxBalanceAnalysisCalculationModel` type](src/Types.jl). Let's build one of these objects for our dual problem and store it in the `dual_model::MyDualFluxBalanceAnalysisCalculationModel` variable. 

> __What's in the dual model?__ The dual model will contain the dual bounds array, the dual objective function array, the dual constraint matrix, and the right-hand side of the dual problem (the objective coefficients of the primal problem). 

Let's build the dual model, and store it in the `dual_model::MyDualFluxBalanceAnalysisCalculationModel` variable, by calling [a `build(...)` method](src/Factory.jl):

In [16]:
# TODO: Build an instance of the dual model using the associated build function
dual_model = nothing; # initialize nothing for the dual model

Let's solve the dual problem by calling [a `solve(...)` method](src/Compute.jl) with the appropriate problem model instance.

> __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method will [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

The [`solve(...)` method](src/Compute.jl) when called with the dual model returns a `dual_solution::Dict{String, Any}` dictionary containing information about the dual solution.

In [17]:
dual_solution = let
    
    solution = nothing; # initialize nothing for the solution
    
    # TODO: Call the solve function with the dual model (don't forget to handle errors with a try-catch block)
    # TODO: Comment out the line below after you call the solve function
    throw(ErrorException("TODO: Call the solve function with the dual model"));

    # return solution
    solution
end;

ErrorException: TODO: Call the solve function with the dual model

Let's extract the dual variable values from the `dual_solution::Dict{String, Any}` dictionary that are non-zero. We store the dual variable data in the `non_zero_dual_data::NamedTuple` variable. 

> __What's in the dual variable NamedTuple?__ We store the non-zero dual variables in the `y`, `p`, and `q` fields of the `non_zero_dual_data::NamedTuple` variable. The indexes of the non-zero dual variables (in the order they appear in the primal problem) are stored in the `i`, `j`, and `k` fields of the `non_zero_dual_data::NamedTuple` variable.

Let's extract the non-zero dual variable data:

In [18]:
non_zero_dual_data = let
    
    # alias the primal model -
    model = primal_model;

    # initialize -
    S = model.S; # get the stoichiometric matrix
    number_of_reactions = size(S,2); # columns |R|
    number_of_metabolites = size(S,1); # rows |M|
    y = zeros(number_of_metabolites); # initialize
    p = zeros(number_of_reactions); # initialize
    q = zeros(number_of_reactions); # initialize
    z = dual_solution["argmin"]; # get the dual variable values

    # process the first |M| dual variables 
    for i ∈ 1:number_of_metabolites
        y[i] = z[i] # y-variables (first |M| dual variables are y-variables)
    end

    # process the next |R| dual variables
    for i ∈ 1:number_of_reactions
        p[i] = z[number_of_metabolites + i] # p-variables (next |R| dual variables are p-variables)
        q[i] = z[number_of_metabolites + number_of_reactions + i] # q-variables (last |R| dual variables are q-variables)
    end
   
    # extract the non-zero dual variable values and their indices
    data = (
        y = findall(!iszero, y) |> i -> y[i], # non-zero y-variables
        p = findall(!iszero, p) |> i -> p[i], # non-zero p-variables
        q = findall(!iszero, q) |> i -> q[i], # non-zero q-variables
        i = findall(!iszero, y), # index of non-zero y-variables
        j = findall(!iszero, p), # index of non-zero p-variables
        k = findall(!iszero, q), # index of non-zero q-variables
    );

    data # return
end;

UndefVarError: UndefVarError: `dual_solution` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Interpret the dual solution data
For this style of primal, the dual solution has the traditional interpretation of measuring the sensitivity of the objective function to changes in the constraints. Thus, we can interpret the dual variables through this lens.

> __Interpretation of dual variables__:
> 
> * __Dual vector__ $\mathbf{y}$: The dual variables associated with the material balance constraints ($\mathbf{y}$) can be interpreted as the shadow prices of the metabolites in the system, i.e., the change in the objective function per unit change in the availability of a metabolite.
> * __Dual vector__ $\mathbf{p}$: The dual variables associated with the upper flux bounds ($\mathbf{p}$) can be interpreted as the sensitivity of the objective function to changes in the upper flux bounds, i.e., the change in the objective function per unit change in the upper flux bound of a reaction.
> * __Dual vector__ $\mathbf{q}$: The dual variables associated with the lower flux bounds ($\mathbf{q}$) can be interpreted as the sensitivity of the objective function to changes in the lower flux bounds, i.e., the change in the objective function per unit change in the lower flux bound of a reaction.

Let's dig into what the dual solution is telling us about our urea cycle model.

#### Binding metabolite constraints
Let's look at the dual variables associated with the material balance constraints ($\mathbf{y}$). Which metabolites have non-zero dual variables (suggesting the constraints are active)?

> __What is an active constraint?__: An active constraint is a constraint that holds as an equality at the optimal solution of an optimization problem. In other words, an active constraint is one that directly influences the optimal solution because it is "binding" or "tight." There is no more wiggle room for the solution to move without violating the constraint.

What does this tell us about the system?

In [19]:
let
   
    # alias the primal model -
    model = primal_model;
    list_of_species = model.species; # get the species list

    # initialize -
    y = non_zero_dual_data.y;
    i = non_zero_dual_data.i;
    df = DataFrame();

    # let's make a table of the non-zero y-variables
    for index = 1:length(y)

        species_index = i[index];
        dual_value = y[index];
        metabolite = list_of_species[species_index];

        row_df = (
            species_index = species_index,
            dual_value = dual_value,
            metabolite = metabolite,
        )

        push!(df, row_df)
    end
    
    # show the table -
    pretty_table(df, backend = :text,
         table_format = TextTableFormat(borders = text_table_borders__compact))
end

UndefVarError: UndefVarError: `non_zero_dual_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [20]:
do_I_see_the_dual_y_table = nothing; # TODO: update this flag to {true | false} if the dual y-variable table is visible

#### Binding flux bounds constraints
Next, let's look at the dual variables associated with the upper and lower flux bounds ($\mathbf{p}$ and $\mathbf{q}$). Which reactions have non-zero dual variables (suggesting the constraints are active)?

> __This is a very important result__. The reactions with non-zero dual variables are the rate-limiting steps in the system. These reactions are controlling the flux through the system and are the bottlenecks in the metabolic network. So if we want to increase the flux through the system, we need to focus on these reactions through our metabolic engineering efforts.

So what do we see?

In [21]:
let

    # alias the primal model -
    model = primal_model;
    list_of_reactions = model.reactions; # get the reactions list
    v = primal_solution["argmax"]; # get the primal flux values

    # initialize -
    p = non_zero_dual_data.p;
    q = non_zero_dual_data.q;
    j = non_zero_dual_data.j;
    k = non_zero_dual_data.k;
    df_p = DataFrame();
    df_q = DataFrame();

    # let's make a table of the non-zero p-variables
    for index = 1:length(p)

        reaction_index = j[index];
        dual_value = p[index];
        reaction = list_of_reactions[reaction_index];

        row_df = (
            reaction_index = reaction_index,
            dual_value = dual_value,
            reaction = reaction,
            flux = v[reaction_index],
            string = rd[reaction],
        )

        push!(df_p, row_df)
    end

    # show the p-variable table -
    println("Non-zero dual p-variables (associated with upper flux bounds):")
    pretty_table(df_p, backend = :text, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact))

    println("\n"); # space

    # let's make a table of the non-zero q-variables
    for index = 1:length(q)

        reaction_index = k[index];
        dual_value = q[index];
        reaction = list_of_reactions[reaction_index];

        row_df = (
            reaction_index = reaction_index,
            dual_value = dual_value,
            reaction = reaction,
            flux = v[reaction_index],
            string = rd[reaction],
        )

        push!(df_q, row_df)
    end

    # show the q-variable table -
    println("Non-zero dual q-variables (associated with lower flux bounds):")
    pretty_table(df_q, backend = :text, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact))

end

UndefVarError: UndefVarError: `non_zero_dual_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [22]:
do_I_see_the_p_and_q_tables = nothing; # TODO: update this flag to {true | false} if the dual p- and q-variable tables are visible

__Wow! This is very cool!__ By analyzing the dual solution, we can identify the rate-limiting step(s) in the system and understand how changes in the flux bounds constraints will affect the objective function. 

While this may seem obvious in our simple model, this is a __powerful technique__ that can be applied to more complex models to identify the rate-limiting steps in the system and understand how changes in the flux bounds constraints will affect the objective function, e.g., in the human metabolic reconstructions (with thousands of reactions):
* [Swainston N, Smallbone K, Hefzi H, Dobson PD, Brewer J, Hanscho M, Zielinski DC, Ang KS, Gardiner NJ, Gutierrez JM, Kyriakopoulos S, Lakshmanan M, Li S, Liu JK, Martínez VS, Orellana CA, Quek LE, Thomas A, Zanghellini J, Borth N, Lee DY, Nielsen LK, Kell DB, Lewis NE, Mendes P. Recon 2.2: from reconstruction to model of human metabolism. Metabolomics. 2016;12:109. doi: 10.1007/s11306-016-1051-4. Epub 2016 Jun 7. PMID: 27358602; PMCID: PMC4896983.](https://pubmed.ncbi.nlm.nih.gov/27358602/)
* [Brunk E, Sahoo S, Zielinski DC, Altunkaya A, Dräger A, Mih N, Gatto F, Nilsson A, Preciat Gonzalez GA, Aurich MK, Prlić A, Sastry A, Danielsdottir AD, Heinken A, Noronha A, Rose PW, Burley SK, Fleming RMT, Nielsen J, Thiele I, Palsson BO. Recon3D enables a three-dimensional view of gene variation in human metabolism. Nat Biotechnol. 2018 Mar;36(3):272-281. doi: 10.1038/nbt.4072. Epub 2018 Feb 19. PMID: 29457794; PMCID: PMC5840010.](https://pubmed.ncbi.nlm.nih.gov/29457794/)

## Discussion Questions
Now that we have solved the primal and dual problems, let's explore some interesting questions about our results.

### Discussion Question 1: Sensitivity Analysis Using Dual Variables
Based on your dual variable analysis, you identified which reactions have non-zero dual variables associated with their upper flux bounds (the **p-variables**). These dual variables represent the sensitivity of the objective function to changes in the maximum reaction velocities ($V_{max}^{\circ}$).

**Question:** If we increased the $V_{max}^{\circ}$ for reaction v₂ (EC 4.3.2.1, argininosuccinate lyase) by 10%, what would you predict happens to the optimal urea production rate? Use the dual variable information to make this prediction and explain your reasoning. Would this reaction be a good target for metabolic engineering to increase urea production?

Run the primal simulation to verify your prediction, report the new optimal urea production rate, and compare it to your prediction.

In [23]:
# TODO: Put your DQ1 answer here

In [24]:
did_I_answer_DQ1 = nothing; # TODO: update this flag to {true | false} if you answered DQ1

### Discussion Question 2: Metabolic Bottleneck Identification
The dual solution provides insights into which constraints are "active" or "binding" at the optimal solution, effectively identifying the bottlenecks in the metabolic network.

**Question:** Which lower bound flux constraint (the **q-variables**) is active in your dual solution? What does this imply about the corresponding reaction in the urea cycle? What happens if you relax this constraint (i.e., decrease the lower bound)? How would this affect the overall flux distribution and urea production?

Run the simulation to verify your prediction, report the new optimal urea production rate, and compare it to your prediction.

In [25]:
# TODO: Put your DQ2 answer here

In [26]:
did_I_answer_DQ2 = nothing; # TODO: update this flag to {true | false} if you answered DQ2

___

## Tests
The code block below shows how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing, etc.

In [ ]:
@testset verbose = true "CHEME 5800 PS3 Test Suite" begin
        
    @testset "Setup" begin
        @test isnothing(primal_model) == false
        @test isnothing(rd) == false
        @test isnothing(reversibility_parameter_dictionary) == false
        @test isnothing(maximum_reaction_velocity_dictionary) == false
        @test isnothing(primal_solution) == false

        @test isempty(reversibility_parameter_dictionary) == false
        @test isempty(maximum_reaction_velocity_dictionary) == false

    end

    @testset "Primal Calculation" begin
        @test isempty(primal_solution) == false
        @test do_I_see_the_primal_flux_table == true
    end

    @testset "Dual Calculation" begin
        @test isnothing(dual_model) == false
        @test isnothing(dual_solution) == false
        @test isempty(dual_solution) == false
        @test do_I_see_the_dual_y_table == true
        @test do_I_see_the_p_and_q_tables == true
    end
    
    @testset "Discussion questions" begin
        @test did_I_answer_DQ1 == true
        @test did_I_answer_DQ2 == true
    end
end;

Primal Calculation: Test Failed at /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/PS3-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y163sZmlsZQ==.jl:17
  Expression: do_I_see_the_primal_flux_table == true
   Evaluated: nothing == true

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:680 [inlined]
 [2] macro expansion
   @ ~/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/PS3-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y163sZmlsZQ==.jl:17 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1709 [inlined]
 [4] macro expansion
   @ ~/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/PS3-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y163sZmlsZQ==.jl:16 [inlined]
 [5] macro expansion
   @ ~/.julia/juliaup/julia-1.11.7

TestSetException: Some tests did not pass: 8 passed, 6 failed, 2 errored, 0 broken.

## Biological References
1. [Al-Otaibi NAS, Cassoli JS, Martins-de-Souza D, Slater NKH, Rahmoune H. Human leukemia cells (HL-60) proteomic and biological signatures underpinning cryo-damage are differentially modulated by novel cryo-additives. Gigascience. 2019 Mar 1;8(3):giy155. doi: 10.1093/gigascience/giy155. PMID: 30535373; PMCID: PMC6394207.](https://pmc.ncbi.nlm.nih.gov/articles/PMC6394207/)
2. [Figarola JL, Weng Y, Lincoln C, Horne D, Rahbar S. Novel dichlorophenyl urea compounds inhibit proliferation of human leukemia HL-60 cells by inducing cell cycle arrest, differentiation and apoptosis. Invest New Drugs. 2012 Aug;30(4):1413-25. doi: 10.1007/s10637-011-9711-8. Epub 2011 Jul 5. PMID: 21728022.](https://pubmed.ncbi.nlm.nih.gov/21728022/)
3. [Caldwell RW, Rodriguez PC, Toque HA, Narayanan SP, Caldwell RB. Arginase: A Multifaceted Enzyme Important in Health and Disease. Physiol Rev. 2018 Apr 1;98(2):641-665. doi: 10.1152/physrev.00037.2016. PMID: 29412048; PMCID: PMC5966718.](https://pmc.ncbi.nlm.nih.gov/articles/PMC5966718/)

___